# Topic modelling LDA part
<br> Just to check what variables are needed to save. This jupyter notebook contains all the code for LDA and emails only


In [2]:
import pandas as pd
import pickle
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models  # the module 'gensim' has renamed to gensim_models
import dill

/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [3]:
# Create a small set of emails (10 000)
"""emails_read_all = pd.read_csv('./csv/emails_df.csv')
enron = emails_read_all.head[:10000]
enron.to_csv('./csv/emails_df_10000.csv', index=False)"""

"emails_read_all = pd.read_csv('./csv/emails_df.csv')\nenron = emails_read_all.head[:10000]\nenron.to_csv('./csv/emails_df_10000.csv', index=False)"

In [4]:
#enron = pd.read_csv('./csv/emails_df.csv')
enron = pd.read_csv('./csv/emails_df_10000.csv')

#enron = pd.read_csv('./csv/emails_df.csv')

## Load all the necessary files


In [5]:

with open("./calc_data/data_lemmatized.pkl", 'rb') as my_pickle:
    data_lemmatized = pickle.load(my_pickle)

with open("./calc_data/data.pkl", 'rb') as my_pickle:
    data = pickle.load(my_pickle)

print(len(data))
print(len(data_lemmatized))

#corpora is a dill file (module cannot be pickled)
with open("./calc_data/corpora.dill", 'rb') as f:
            my_corpora = dill.load(f)


240079
240079


In [6]:
#TAKE A SAMPLE OF DATA

enron = enron[:1000]
data_lemmatized = data_lemmatized[:1000]
data = data[:1000]


In [7]:
len(data_lemmatized)


1000

In [8]:
#emails_lemmatized = pd.DataFrame(data_lemmatized, columns=['lemmatized_subject'])
new_column = pd.Series(data_lemmatized)
enron['body_lem'] = new_column

In [9]:
print(data_lemmatized[1])

['travel', 'business', 'meeting', 'take', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'suggest', 'hold', 'business', 'plan', 'meeting', 'take', 'trip', 'formal', 'business', 'meeting', 'even', 'try', 'get', 'honest', 'opinion', 'trip', 'even', 'desire', 'necessary', 'far', 'business', 'meeting', 'think', 'productive', 'try', 'stimulate', 'discussion', 'different', 'group', 'work', 'often', 'presenter', 'speak', 'other', 'quiet', 'wait', 'turn', 'meeting', 'well', 'hold', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'play', 'golf', 'rent', 'boat', 'jet', 'ski', 'fly', 'somewhere', 'take', 'much', 'time']


In [10]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = my_corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


print(type(my_corpora))

<class 'module'>


In [11]:
id2word.keys()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [12]:
mallet_path = './mallet-2.0.8/bin/mallet'

In [13]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
print((lda_model))

LdaModel(num_terms=7610, num_topics=2, decay=0.5, chunksize=100)


In [16]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [17]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(0, '0.028*"image" + 0.009*"buy" + 0.006*"http" + 0.006*"email" + 0.006*"free" + 0.005*"get" + 0.005*"click" + 0.005*"phillip" + 0.005*"information" + 0.005*"make"'), (1, '0.103*"pm" + 0.035*"ect" + 0.015*"request" + 0.015*"hou" + 0.013*"ee" + 0.012*"image" + 0.010*"mail" + 0.010*"d" + 0.010*"i" + 0.008*"showtime"')]


In [18]:
doc_lda = lda_model[corpus]

In [19]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.473652570568314


In [20]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3873081416192695


In [21]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of impor

In [22]:
print(lda_model.show_topic(0))

[('image', 0.02788555), ('buy', 0.008953995), ('http', 0.0064281523), ('email', 0.005540428), ('free', 0.0055325357), ('get', 0.0052934675), ('click', 0.005263495), ('phillip', 0.0050462065), ('information', 0.005008495), ('make', 0.00500212)]


In [23]:
pyLDAvis.display(vis)

## How to get top emails per topic?
There are actually two cases:
<br> a. Best email to given topic
<br> b. Dominant topic for a given email


In [24]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()


    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        #print("i: ", i)
        #print("row: ", row)
        #print("row[1]: ", row[1])
        row = sorted(
            row[0], key=lambda x: (x[1]), 
            reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [25]:
print(type(lda_model[corpus]))
print(len(lda_model[corpus]))
print("corpus type:", type(corpus))
print("corpus len", len(corpus))

<class 'gensim.interfaces.TransformedCorpus'>
1000
corpus type: <class 'list'>
corpus len 1000


In [26]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)



/var/folders/6r/0sxxkqln2g73tp73l2p7xx200000gn/T/ipykernel_17748/825188672.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/var/folders/6r/0sxxkqln2g73tp73l2p7xx200000gn/T/ipykernel_17748/825188672.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [27]:
with open(f'./csv/df_topic_sents_keywords.pkl', 'wb') as my_pickle:
                pickle.dump(data_lemmatized, my_pickle)

In [28]:

print(df_topic_sents_keywords.shape)
print(len(data))
print(len(data_lemmatized))
df_topic_sents_keywords.head()


(1000, 4)
1000
1000


,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s..."
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.


In [29]:
df_topic_sents_keywords['Dominant_Topic'].value_counts()

0    767
1    233
Name: Dominant_Topic, dtype: int64

In [30]:
topic_9 = df_topic_sents_keywords[df_topic_sents_keywords['Dominant_Topic'] == 1].sort_values(by=['Perc_Contribution'], ascending=False)
topic_9.head()
topic_9.rename(columns={0: 'Text'}, inplace=True)
topic_9.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
888,1,0.9996,"pm, ect, request, hou, ee, image, mail, d, i, ...",Amazon.com: Weekly Movie Showtimes [IMAGE]\n...
683,1,0.9996,"pm, ect, request, hou, ee, image, mail, d, i, ...",\n=09=09=09=09[IMAGE] =09=09\n[IMAGE]=09[IMAGE...
780,1,0.9996,"pm, ect, request, hou, ee, image, mail, d, i, ...",\n=09=09=09=09[IMAGE] =09=09\n[IMAGE]=09[IMAGE...
485,1,0.9994,"pm, ect, request, hou, ee, image, mail, d, i, ...",---------------------- Forwarded by Phillip K ...
972,1,0.9993,"pm, ect, request, hou, ee, image, mail, d, i, ...",[IMAGE]\n[IMAGE]=09=09=09=09=09Showtimes start...


In [31]:
#messages = topic_9[0:10]['Text'].to_list()
#print(type(messages))
#for msg in messages:
#    print(msg)




In [32]:


with open('./calc_data/enron_df.pkl', 'rb') as enron_file:
    enron = pickle.load(enron_file)

enron = enron[:1000]
print(enron.shape)
enron[['Message-ID', 'content']]
#row 860 from enron dataframe
enron[779:782]['content']



(1000, 18)


1340    If you cannot read this email, please click he...
1341    \n=09=09=09=09[IMAGE] =09=09\n[IMAGE]=09[IMAGE...
1342    The Ultimate Traditional & Internet Marketing ...
Name: content, dtype: object

In [33]:
df_topic_sents_keywords['email_content'] = enron['content']

## PLAN TO DO THE HIERARCHY
It seems that the order in this list is still intact. We can use that. 
<br> Join few columns from enron dataframe to this result.
<br> 
<br> Subtopic calculation:
<br> What information we need to calculate next level LDA?
<br> What needs to be recalculated Corpora etc?


In [34]:
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0,email_content
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n,Here is our forecast\n\n
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!,test successful. way to go!!!
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s..."
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.


In [35]:
print(type(enron))
enron.head()


<class 'pandas.core.frame.DataFrame'>


,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [36]:
df_topic_sents_keywords['ID'] = enron['Message-ID']
df_topic_sents_keywords['Subject'] = enron['Subject']
df_topic_sents_keywords['enron_content'] = enron['content']
df_topic_sents_keywords['From'] = enron['From']
df_topic_sents_keywords['To'] = enron['To']
df_topic_sents_keywords['Cc'] = enron['X-cc']
df_topic_sents_keywords['Bcc'] = enron['X-bcc']
df_topic_sents_keywords.head()


,Dominant_Topic,Perc_Contribution,Topic_Keywords,0,email_content,ID,Subject,enron_content,From,To,Cc,Bcc
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n,Here is our forecast\n\n,<18782981.1075855378110.JavaMail.evans@thyme>,NaN,Here is our forecast\n\n,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,NaN
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,<15464986.1075855378456.JavaMail.evans@thyme>,Re:,Traveling to have a business meeting takes the...,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),NaN,NaN
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!,test successful. way to go!!!,<24216240.1075855687451.JavaMail.evans@thyme>,Re: test,test successful. way to go!!!,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),NaN,NaN
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...",<13505866.1075863688222.JavaMail.evans@thyme>,NaN,"Randy,\n\n Can you send me a schedule of the s...",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,NaN
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,<30922949.1075863688243.JavaMail.evans@thyme>,Re: Hello,Let's shoot for Tuesday at 11:45.,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),NaN,NaN


In [37]:
print(type(data_lemmatized))
len(data_lemmatized)
print(data_lemmatized[0])

<class 'list'>
['forecast']


In [38]:
df_topic_sents_keywords['data_lemmatized'] = data_lemmatized
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0,email_content,ID,Subject,enron_content,From,To,Cc,Bcc,data_lemmatized
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n,Here is our forecast\n\n,<18782981.1075855378110.JavaMail.evans@thyme>,NaN,Here is our forecast\n\n,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,NaN,[forecast]
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,<15464986.1075855378456.JavaMail.evans@thyme>,Re:,Traveling to have a business meeting takes the...,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),NaN,NaN,"[travel, business, meeting, take, fun, trip, e..."
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!,test successful. way to go!!!,<24216240.1075855687451.JavaMail.evans@thyme>,Re: test,test successful. way to go!!!,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),NaN,NaN,"[test, successful, way, go]"
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...",<13505866.1075863688222.JavaMail.evans@thyme>,NaN,"Randy,\n\n Can you send me a schedule of the s...",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,NaN,"[schedule, salary, level, scheduling, group, t..."
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,<30922949.1075863688243.JavaMail.evans@thyme>,Re: Hello,Let's shoot for Tuesday at 11:45.,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),NaN,NaN,"[let, shoot]"


In [39]:
with open('./csv/tmp_sample.pkl', 'wb') as my_pickle:
    pickle.dump(df_topic_sents_keywords, my_pickle)

## WHAT's NEEDED FOR LDA (Level X)

<br> corpus
<br> id2word
<br>
<br> id2word: data_lemmatized, corpora.Didtionary(data_lemmatized)
<br> corpus: id2word, data_lemmatized

In [42]:
#print(type(texts))
#df_topic_sents_keywords.drop('topic_list', axis=1, inplace=True)

In [43]:
df_topic_sents_keywords.head()

#df_topic_sents_keywords['topic_hier'] = empty_list

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0,email_content,ID,Subject,enron_content,From,To,Cc,Bcc,data_lemmatized
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n,Here is our forecast\n\n,<18782981.1075855378110.JavaMail.evans@thyme>,NaN,Here is our forecast\n\n,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,NaN,[forecast]
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,<15464986.1075855378456.JavaMail.evans@thyme>,Re:,Traveling to have a business meeting takes the...,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),NaN,NaN,"[travel, business, meeting, take, fun, trip, e..."
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!,test successful. way to go!!!,<24216240.1075855687451.JavaMail.evans@thyme>,Re: test,test successful. way to go!!!,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),NaN,NaN,"[test, successful, way, go]"
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...",<13505866.1075863688222.JavaMail.evans@thyme>,NaN,"Randy,\n\n Can you send me a schedule of the s...",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,NaN,"[schedule, salary, level, scheduling, group, t..."
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,<30922949.1075863688243.JavaMail.evans@thyme>,Re: Hello,Let's shoot for Tuesday at 11:45.,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),NaN,NaN,"[let, shoot]"


In [44]:
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0,email_content,ID,Subject,enron_content,From,To,Cc,Bcc,data_lemmatized
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n,Here is our forecast\n\n,<18782981.1075855378110.JavaMail.evans@thyme>,NaN,Here is our forecast\n\n,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,NaN,[forecast]
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,<15464986.1075855378456.JavaMail.evans@thyme>,Re:,Traveling to have a business meeting takes the...,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),NaN,NaN,"[travel, business, meeting, take, fun, trip, e..."
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!,test successful. way to go!!!,<24216240.1075855687451.JavaMail.evans@thyme>,Re: test,test successful. way to go!!!,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),NaN,NaN,"[test, successful, way, go]"
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...",<13505866.1075863688222.JavaMail.evans@thyme>,NaN,"Randy,\n\n Can you send me a schedule of the s...",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,NaN,"[schedule, salary, level, scheduling, group, t..."
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,<30922949.1075863688243.JavaMail.evans@thyme>,Re: Hello,Let's shoot for Tuesday at 11:45.,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),NaN,NaN,"[let, shoot]"


In [45]:
old_values = df_topic_sents_keywords['topic_hier'].to_list()
print(old_values)

KeyError: 'topic_hier'

In [46]:
df = df_topic_sents_keywords.copy()



def convert_to_list(value):
    return [value]

df['topic_hier'] = df['Dominant_Topic'].apply(convert_to_list)

df.head()



,Dominant_Topic,Perc_Contribution,Topic_Keywords,0,email_content,ID,Subject,enron_content,From,To,Cc,Bcc,data_lemmatized,topic_hier
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n,Here is our forecast\n\n,<18782981.1075855378110.JavaMail.evans@thyme>,NaN,Here is our forecast\n\n,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,NaN,[forecast],[0]
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,<15464986.1075855378456.JavaMail.evans@thyme>,Re:,Traveling to have a business meeting takes the...,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),NaN,NaN,"[travel, business, meeting, take, fun, trip, e...",[0]
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!,test successful. way to go!!!,<24216240.1075855687451.JavaMail.evans@thyme>,Re: test,test successful. way to go!!!,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),NaN,NaN,"[test, successful, way, go]",[0]
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...",<13505866.1075863688222.JavaMail.evans@thyme>,NaN,"Randy,\n\n Can you send me a schedule of the s...",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,NaN,"[schedule, salary, level, scheduling, group, t...",[0]
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,<30922949.1075863688243.JavaMail.evans@thyme>,Re: Hello,Let's shoot for Tuesday at 11:45.,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),NaN,NaN,"[let, shoot]",[0]


In [47]:
new_value = df['Dominant_Topic'].to_list()
old_value = df['topic_hier'].to_list()

print(type(new_value))

for i, old in enumerate(old_value):
    old.append(new_value[i])
    old_value[i] = old
df['topic_hier'] = old_value
df

<class 'list'>


,Dominant_Topic,Perc_Contribution,Topic_Keywords,0,email_content,ID,Subject,enron_content,From,To,Cc,Bcc,data_lemmatized,topic_hier
0,0,0.9300,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n,Here is our forecast\n\n,<18782981.1075855378110.JavaMail.evans@thyme>,NaN,Here is our forecast\n\n,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,NaN,[forecast],"[0, 0]"
1,0,0.9986,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,<15464986.1075855378456.JavaMail.evans@thyme>,Re:,Traveling to have a business meeting takes the...,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),NaN,NaN,"[travel, business, meeting, take, fun, trip, e...","[0, 0]"
2,0,0.9780,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!,test successful. way to go!!!,<24216240.1075855687451.JavaMail.evans@thyme>,Re: test,test successful. way to go!!!,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),NaN,NaN,"[test, successful, way, go]","[0, 0]"
3,0,0.9915,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...",<13505866.1075863688222.JavaMail.evans@thyme>,NaN,"Randy,\n\n Can you send me a schedule of the s...",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,NaN,"[schedule, salary, level, scheduling, group, t...","[0, 0]"
4,0,0.9595,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,<30922949.1075863688243.JavaMail.evans@thyme>,Re: Hello,Let's shoot for Tuesday at 11:45.,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),NaN,NaN,"[let, shoot]","[0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0.9948,"image, buy, http, email, free, get, click, phi...",There are three other deals that I will fax to...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[deal, fax, let, know, interest, thank, drive,...","[0, 0]"
996,0,0.9966,"image, buy, http, email, free, get, click, phi...","\nPhillip,\n \nCould you please do me a favor?...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[please, favor, like, read, current, title, po...","[0, 0]"
997,1,0.9530,"pm, ect, request, hou, ee, image, mail, d, i, ...","Phillip,\n\nPursuant to your request, please s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[pursuant, request, see, attach, thank, renee]","[1, 1]"
998,1,0.8416,"pm, ect, request, hou, ee, image, mail, d, i, ...","<<3MMP10!.DOC>>\nPhillip,\n\nEnclosed please ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[enclose, find, amendment, contract, review, e...","[1, 1]"


In [48]:
print(len(df))
my_lem = df[0].to_list
print(my_lem)

1000
<bound method IndexOpsMixin.tolist of 0                              Here is our forecast\n\n 
1      Traveling to have a business meeting takes the...
2                         test successful.  way to go!!!
3      Randy,\n\n Can you send me a schedule of the s...
4                    Let's shoot for Tuesday at 11:45.  
                             ...                        
995    There are three other deals that I will fax to...
996    \nPhillip,\n \nCould you please do me a favor?...
997    Phillip,\n\nPursuant to your request, please s...
998     <<3MMP10!.DOC>>\nPhillip,\n\nEnclosed please ...
999     <<3MMPRED.DOC>>\nPhillip,\n\nEnclosed please ...
Name: 0, Length: 1000, dtype: object>


In [52]:
df_2 = df[['Dominant_Topic', 'Perc_Contribution']][0:5]
df_3 = df[['Dominant_Topic', 'Topic_Keywords', 'email_content']][0:5]
df_4 = pd.concat([df_2, df_3], axis=1)
df_4.head(10)


,Dominant_Topic,Perc_Contribution,Dominant_Topic,Topic_Keywords,email_content
0,0,0.9300,0,"image, buy, http, email, free, get, click, phi...",Here is our forecast\n\n
1,0,0.9986,0,"image, buy, http, email, free, get, click, phi...",Traveling to have a business meeting takes the...
2,0,0.9780,0,"image, buy, http, email, free, get, click, phi...",test successful. way to go!!!
3,0,0.9915,0,"image, buy, http, email, free, get, click, phi...","Randy,\n\n Can you send me a schedule of the s..."
4,0,0.9595,0,"image, buy, http, email, free, get, click, phi...",Let's shoot for Tuesday at 11:45.


# TEsting


In [ ]:
print(type(data_lemmatized))
print(data_lemmatized[1])


<class 'list'>
['travel', 'business', 'meeting', 'take', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'suggest', 'hold', 'business', 'plan', 'meeting', 'take', 'trip', 'formal', 'business', 'meeting', 'even', 'try', 'get', 'honest', 'opinion', 'trip', 'even', 'desire', 'necessary', 'far', 'business', 'meeting', 'think', 'productive', 'try', 'stimulate', 'discussion', 'different', 'group', 'work', 'often', 'presenter', 'speak', 'other', 'quiet', 'wait', 'turn', 'meeting', 'well', 'hold', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'play', 'golf', 'rent', 'boat', 'jet', 'ski', 'fly', 'somewhere', 'take', 'much', 'time']
